In [1]:
import pandas as pd
import numpy as np
import pickle
import torch
import torch.nn as nn
from torch.nn import functional as F
from model import *

In [2]:
# import training and validation data (encoded data)
train_data = torch.tensor(np.fromfile('data/train.bin', dtype=np.int16), dtype=torch.long)
val_data = torch.tensor(np.fromfile('data/val.bin', dtype=np.int16), dtype=torch.long)

In [3]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [6]:
%%time

model = GPTLanguageModel()
m = model.to(device)

# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

num_steps = 1000
for step in range(num_steps):

    # every once in a while evaluate the loss on train and val sets
    if step % eval_interval == 0 or step == num_steps - 1:
        losses = estimate_loss()
        print(f"step {step}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

1.216384 M parameters
step 0: train loss 4.9924, val loss 4.9957
step 100: train loss 2.1849, val loss 2.1959
step 200: train loss 2.0204, val loss 2.0281
step 300: train loss 1.9340, val loss 1.9511
step 400: train loss 1.8781, val loss 1.8959
step 500: train loss 1.8440, val loss 1.8621
step 600: train loss 1.8219, val loss 1.8460
step 700: train loss 1.8031, val loss 1.8284
step 800: train loss 1.7789, val loss 1.8096
step 900: train loss 1.7666, val loss 1.7937
step 999: train loss 1.7535, val loss 1.7806
Wall time: 2min 45s


In [7]:
# save model
torch.save(m, 'data/SmallMusicModel.pth')